In [ ]:
from xarm.wrapper import XArmAPI
from applied_planning.hardware import XArmLite6Adapter  

ROBOT_IP = "192.168.1.161"
arm = XArmAPI(ROBOT_IP)
robot = XArmLite6Adapter(ROBOT_IP)

ROBOT_IP: 192.168.1.161, VERSION: v1.11.4, PROTOCOL: V1, DETAIL: 6,9,LI1003,DL1000,v1.11.4, TYPE1300: [0, 0]
change protocol identifier to 3


ControllerError, code: 2


In [9]:
robot.reset() # move robot to home position

[set_state], xArm is ready to move
Moving to home position...
✓ Home position reached


{'qpos': array([-0.      ,  1.444992,  2.661739,  0.      ,  1.216747,  0.      ]),
 'qvel': array([0., 0., 0., 0., 0., 0.])}

ControllerError had cleanControllerError had clean
ControllerError had clean



In [ ]:
import time
import math
import numpy as np

print("Moving robot in circular pattern with sinusoidal vertical motion...")
print("Press Ctrl+C to stop")

# CORRECTED Parameters - much smaller and closer
radius = 0.08  # 8cm radius (was 15cm - too big!)
center_x = 0.25  # 25cm in front (was 30cm+15cm = 45cm - too far!)
center_y = 0.0  # Centered
base_z = 0.15  # 15cm height (was 20cm - may be too low for some positions)
z_amplitude = 0.03  # 3cm vertical wave (was 5cm)

duration = 30
num_points = 60  # Fewer points = smoother motion
speed = 50  # Slower speed for safety (was 100)

try:
    # Get current position
    code, current_pos = robot.arm.get_position()
    print(f"Current position: {current_pos[:3]}")
    
    # Enable motion
    robot.arm.set_mode(0)
    robot.arm.set_state(0)
    time.sleep(0.5)
    
    # Move to starting position first
    start_x = (center_x + radius) * 1000
    start_y = center_y * 1000
    start_z = base_z * 1000
    
    print(f"Moving to start position: ({start_x:.1f}, {start_y:.1f}, {start_z:.1f})mm")
    code = robot.arm.set_position(
        x=start_x, y=start_y, z=start_z,
        roll=180, pitch=0, yaw=0,
        speed=30, wait=True
    )
    
    if code != 0:
        print(f"ERROR: Could not reach start position! Code={code}")
        print("Try adjusting center_x, center_y, or base_z")
        raise ValueError(f"Cannot reach start position, error code: {code}")
    
    print("Starting circular motion...")
    
    for i in range(num_points):
        # Calculate positions
        angle = 2 * math.pi * i / num_points
        z_angle = 4 * math.pi * i / num_points
        
        x = (center_x + radius * math.cos(angle)) * 1000
        y = (center_y + radius * math.sin(angle)) * 1000
        z = (base_z + z_amplitude * math.sin(z_angle)) * 1000
        
        # Move to position
        code = robot.arm.set_position(
            x=x, y=y, z=z,
            roll=180, pitch=0, yaw=0,
            speed=speed,
            wait=False  # Don't wait for smooth motion
        )
        
        if code != 0:
            print(f"WARNING: Point {i} unreachable, code={code}, pos=({x:.1f},{y:.1f},{z:.1f})")
            # Continue anyway - robot will skip unreachable points
        
        if i % 10 == 0:
            print(f"Progress: {i}/{num_points}")
        
        time.sleep(duration / num_points)
    
    print("\n✓ Circular motion complete!")
    
    # Return to safe position
    print("Returning to home...")
    robot.arm.set_position(x=300, y=0, z=200, roll=180, pitch=0, yaw=0, speed=50, wait=True)
    
except KeyboardInterrupt:
    print("\nStopped by user")
    robot.arm.set_state(4)
    
except Exception as e:
    print(f"\nError: {e}")
    robot.arm.set_state(4)

Generating smooth circular trajectory with sine wave...
Generated 100 waypoints
First point: [449.99999999999994, 0.0, 200.0]
Last point: [449.7040092642407, -9.418577929396989, 193.7333383217848]
Waypoint 0/100
Waypoint 10/100
ControllerError, code: 21ControllerError, code: 21

ControllerError, code: 21
[SDK][ERROR][2025-11-02 21:52:59][base.py:383] - - API -> set_position -> code=1, pos=[409.34529411321176, 102.68206588930329, 249.9013364214136, 3.141592653589793, 0.0, 0.0], radius=-1, velo=100.0, acc=2000
[SDK][ERROR][2025-11-02 21:53:00][base.py:383] - - API -> set_position -> code=1, pos=[402.68206588930326, 109.34529411321172, 249.9013364214136, 3.141592653589793, 0.0, 0.0], radius=-1, velo=100.0, acc=2000
[SDK][ERROR][2025-11-02 21:53:00][base.py:383] - - API -> set_position -> code=1, pos=[395.6135984623034, 115.57698641636837, 249.11436253643447, 3.141592653589793, 0.0, 0.0], radius=-1, velo=100.0, acc=2000
[SDK][ERROR][2025-11-02 21:53:00][base.py:383] - - API -> set_position